In [43]:
import torch

import numpy as np

---

![title](images/vocab_one_hot.png)

In [44]:
# vocab of one-hot vectors
vocab_size = 7
vocab = np.eye(vocab_size)

print(vocab)
print(vocab.shape)

[[1. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 0. 0. 1.]]
(7, 7)


---

M is a T-th order lower triangular matrix

![title](images/matrix.png)

In [13]:
# forgetting factor (alpha) --> ]0,1[
ff = 0.5

In [51]:
M1 = torch.tensor([[1]
                  ], dtype=torch.float64)

M2 = torch.tensor([[1,  0],
                   [ff, 1]
                  ], dtype=torch.float64)

M3 = torch.tensor([[1,     0,     0],
                   [ff**1, 1,     0],
                   [ff**2, ff**1, 1],
                  ], dtype=torch.float64)

M4 = torch.tensor([[1,     0,     0,     0],
                   [ff**1, 1,     0,     0],
                   [ff**2, ff**1, 1,     0],
                   [ff**3, ff**2, ff**1, 1],
                  ], dtype=torch.float64)

M5 = torch.tensor([[1,     0,     0,     0,     0],
                   [ff**1, 1,     0,     0,     0],
                   [ff**2, ff**1, 1,     0,     0],
                   [ff**3, ff**2, ff**1, 1,     0],
                   [ff**4, ff**3, ff**2, ff**1, 1],
                  ], dtype=torch.float64)

M6 = torch.tensor([[1,     0,     0,     0,     0,     0],
                   [ff**1, 1,     0,     0,     0,     0],
                   [ff**2, ff**1, 1,     0,     0,     0],
                   [ff**3, ff**2, ff**1, 1,     0,     0],
                   [ff**4, ff**3, ff**2, ff**1, 1,     0],
                   [ff**5, ff**4, ff**3, ff**2, ff**1, 1],
                  ], dtype=torch.float64)
print(M6)

tensor([[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.5000, 1.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2500, 0.5000, 1.0000, 0.0000, 0.0000, 0.0000],
        [0.1250, 0.2500, 0.5000, 1.0000, 0.0000, 0.0000],
        [0.0625, 0.1250, 0.2500, 0.5000, 1.0000, 0.0000],
        [0.0312, 0.0625, 0.1250, 0.2500, 0.5000, 1.0000]], dtype=torch.float64)


---

## FOFE encoding
A sentence of 2 words: the second line of table 3.2 above

![title](images/partial_encoding.png)

In [54]:
# V is a matrix arranging all one-hot codes of the words in the sentence row by row.
# For sentence {w6, w4}, V = [e6, e4] => [[0., 0., 0., 0., 0., 0., 1.], [0., 0., 0., 0., 1., 0., 0.]]
V_w6w4 = torch.tensor([vocab[6], vocab[4]], dtype=torch.float64)
print(V_w6w4)
print(V_w6w4.shape)

tensor([[0., 0., 0., 0., 0., 0., 1.],
        [0., 0., 0., 0., 1., 0., 0.]], dtype=torch.float64)
torch.Size([2, 7])


In [62]:
# Multiply matrices to get the FOFE encoding
S = M2.mm(V_w6w4)
print(S)

tensor([[0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 1.0000],
        [0.0000, 0.0000, 0.0000, 0.0000, 1.0000, 0.0000, 0.5000]],
       dtype=torch.float64)


Each row vector of S represents a FOFE code of the partial sequence up to each position in the sentence.

In [64]:
# So, the FOFE Encoding for [word6, word4] is the last row of S
print(f"FOFE for sentence (word6, word4) is:\n{S[-1]}")

FOFE for sentence (word6, word4) is:
tensor([0.0000, 0.0000, 0.0000, 0.0000, 1.0000, 0.0000, 0.5000],
       dtype=torch.float64)


---

## Compute another FOFE encoding
A sentence of 6 words: the last line of table 3.2 above

In [65]:
V_645054 = torch.tensor([vocab[6], vocab[4], vocab[5], vocab[0], vocab[5], vocab[4]], dtype=torch.float64)
print(V_645054.shape)

torch.Size([6, 7])


In [66]:
S = M6.mm(V_645054)
print(f"FOFE for sentence (word6, word4, word5, word0, word5, word4) is:\n{S[-1]}")

FOFE for sentence (word6, word4, word5, word0, word5, word4) is:
tensor([0.2500, 0.0000, 0.0000, 0.0000, 1.0625, 0.6250, 0.0312],
       dtype=torch.float64)


In [72]:
expected_fofe = torch.tensor([ff**2, 0, 0, 0, 1+ff**4, ff+ff**3, ff**5], dtype=torch.float64)
print(f"Are computed FOFE and Expected FOFE equal --> {expected_fofe.equal(S[-1])}")

Are computed FOFE and Expected FOFE equal --> True


---

## Pre-compute all M matrices

![title](images/matrix.png)

In [89]:
def compute_m_matrix(forgetting_factor, order):
    """
    Arguments
        forgetting_factor - float, forgetting factor (alpha value) between 0 and 1 exclusively
        order - integer, the order of the lower triangular matrix
    Returns
        numpy array, the lower triangular matrix of shape (order, order)
    """
    # starts with an all zero matrix of the correct size
    m = np.zeros((order, order))
    
    # fill the lower triangle with power of the forgetting factor (ff)
    for c in range(order):       # loop over each columns
        p = 0                    # reset power value
        for r in range(order):   # loop over each rows
            if r >= c:
                m[r,c] = forgetting_factor**p
                p += 1
    return m

In [97]:
# T is the higher order of the matrix, 
# this is also the max number of token per sentence we will accept
T = 50
M = []
for t in range(T):
    M.append(compute_m_matrix(forgetting_factor=0.5, order=t))

---

Compare the computed matrix for order 6 to the one created manually at the top of this notebook

In [101]:
M[6]

array([[1.     , 0.     , 0.     , 0.     , 0.     , 0.     ],
       [0.5    , 1.     , 0.     , 0.     , 0.     , 0.     ],
       [0.25   , 0.5    , 1.     , 0.     , 0.     , 0.     ],
       [0.125  , 0.25   , 0.5    , 1.     , 0.     , 0.     ],
       [0.0625 , 0.125  , 0.25   , 0.5    , 1.     , 0.     ],
       [0.03125, 0.0625 , 0.125  , 0.25   , 0.5    , 1.     ]])

In [109]:
(M6.numpy() == M[6]).all()

True